# Supervised Learning Example: Classification of Signal and Background of a search for heavy resonance

<br>
<br>

## 1. Download the Data
For illustration we will use data from the $X \rightarrow ZV \rightarrow \ell\ell qq$ (V = W,Z) in the heavy Higgs scenario production through gluon-gluon fusion (ggF). We'll use backgrounds of Top, Diboson and Z+jets events.

<img src="Feynman-all.png" width="600">

Root files of signal and background processes you can find here: [LLqqFiles](https://drive.google.com/open?id=1MPzf01CJx8kbix0ko30SbYO7xlUd4u1W)

Download the zip file and put them all under a `Files/` directory. The zip file contains:

```python
Files/Top.root
Files/Diboson.root
Files/Zjets.root
Files/ggH1000.root # signal of heavy Higgs with M=1TeV
```

Each root file contains a TTree with name `Nominal` filled with a set of variables filled during offline analysis. Some preselection has already been done. Each event here contains exactly 2 leptons and 1 FatJet. 

<br>
<br>

## 2. Load data

In [ ]:
# import the libraries to use later
import uproot
import numpy as np
import pandas as pd

![](logo-uproot.png)

We will read the root file and covert it to a Pandas DataFrame using **uproot**. Uproot is a software package for I/O methods between ROOT and numpy and pandas. There is a huge number of functionalities for ROOT and Pandas file manipulation. Check-out this link: [Uproot page](https://github.com/scikit-hep/uproot)

In [ ]:
# Lets have a look at the number of events of each file

inputFiles = ['Top','Zjets','Diboson','ggH1000']

for i in inputFiles:
    inFile = 'Files/'+i+'.root'
    theFile = uproot.open(inFile)
    tree = uproot.open(inFile)['Nominal']
    Nevents = uproot.open(inFile)['Nominal'].numentries
    print ('Number of events in '+inFile,'\t'+str(Nevents))
    

In [ ]:
# convert all of them to Pandas Dataframes
DF_Signal  = uproot.open('Files/ggH1000.root')['Nominal'].pandas.df()
DF_Top     = uproot.open('Files/Top.root')['Nominal'].pandas.df()
DF_Zjets   = uproot.open('Files/Zjets.root')['Nominal'].pandas.df()
DF_Diboson = uproot.open('Files/Diboson.root')['Nominal'].pandas.df()

In [ ]:
pd.set_option('display.max_columns', 40)
DF_Top.head()
# print(DF_Top.head())
# print(DF_Zjets.head())
# print(DF_Diboson.head())

In [ ]:
#Lets print few events to see the content
DF_Signal.head()

<br>
<br>


## 3. Variable Description

Basic info from both leptons and the FatJet. ( 𝑝𝑇 , 𝜂 , 𝜙 , 𝐸 , lepton charge)

Dilepton 𝑝𝑇  and mass

More advanced features for the FatJet. Jet substructure variables  𝐷2,𝐶2 

**FullEventWeight** is the generator MC weight * all other Scale Factor weights used in the llqq analysis

**isSignal** in an ``int`` which declares whether the event is signal(=1) or backround (=0). We'll use this variable later on as label for each event in the DNN training and testing process. 

**Note**: if this kind of variable is missing from your initial .root file then it's still possible to add it with DF or NumPy

<br>
<br>

## 4. Visualizing input data

Visualization of inputs and results can be quickly done with [**Matplotlib**](https://matplotlib.org/)

Lets try to plot the variables in the DFs for signal and background

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

VariablesToPlot = ['lep1_pt','lep2_pt','lep1_E','lep2_E','Zll_mass','Zll_pt','MET','fatjet_pt']

for var in VariablesToPlot:
    #adopt a common binning scheme for all channels
    bins = np.linspace(min(DF_Top[var]), max(DF_Top[var]) + 1, 100)
    
    plt.hist(DF_Top[var], histtype='step', normed=True, bins=bins, label='Top', linewidth=2)
    plt.hist(DF_Signal[var], histtype='step', normed=True, bins=bins, label='ggFH', linewidth=2)
    plt.hist(DF_Diboson[var], histtype='step', normed=True, bins=bins, label='Diboson', linewidth=2)
    plt.hist(DF_Zjets[var], histtype='step', normed=True, bins=bins, label='Z+jets', linewidth=2)
    
    plt.xlabel(var)
#     plt.yscale('log')
    plt.legend(loc='best')
    plt.show()

<br>
<br>

### 5. Preparing your data as inputs to a DNN

We have already converted the .root files for signal and background to Pandas DFs. You can then use these for the rest of the lecture or you can save to disk in DataFrame or other format if you want to start from were you've left off.

Saving to disk you can do with the `to_pickle(filename.pkl)` command like:

In [ ]:
# Saving DF to disk
DF_Top.to_pickle('Files/Top_DF.pkl')
DF_Diboson.to_pickle('Files/Diboson_DF.pkl')
DF_Zjets.to_pickle('Files/Zjets_DF.pkl')
DF_Signal.to_pickle('Files/ggH1000_DF.pkl')

You can then read back your .pkl files with commands like:

```python
df_tmp = pd.read_pickle('Files/Diboson_DF.pkl')
```

<br>
<br>

### 6. Prepare signal and background inputs and shuffle your data

For simplicity we are going to have two main classes to ask the DNN to make predictions. One Signal and one Background class. If you wish, you can make this a multi-classification problem by introducing a separate class for each background source but for the moment we are keeping this as simple as possible.

Lets add all background sources together in one DF file. This you can do with the `pd.concat` command as below:

In [ ]:
import sklearn.utils
def Shuffling(df):
    df = sklearn.utils.shuffle(df,random_state=123) #'123' is the random seed
    df = df.reset_index(drop=True)# drop=True does not allow the reset_index to insert a new column with the suffled index
    return df

# Now lets shuffle the events in each file
DF_Signal = Shuffling(DF_Signal)
DF_Zjets  = Shuffling(DF_Zjets)
DF_Top    = Shuffling(DF_Top)
DF_Diboson= Shuffling(DF_Diboson)

In [ ]:
# Put all the files in lists
signalList = [DF_Signal[:50000] ] #just 1 file in this list but depending on your input you might want to extend this.
backgroundList = [DF_Top[:17000],DF_Diboson[:17000],DF_Zjets[:17000]] # 3 files in the background list

In [ ]:
# check how many files are there
len(backgroundList)

In [ ]:
# Lets add them together
totalPD_sig = pd.concat(signalList,ignore_index=True)
totalPD_bkg = pd.concat(backgroundList,ignore_index=True) #add all channels of background together to get 1 DF

#print the number of event in each
print ('Signal events:',totalPD_sig.shape)
print ('Background events:',totalPD_bkg.shape)

In [ ]:
# add together signal+background PDs
df_total = pd.concat([totalPD_sig,totalPD_bkg],ignore_index=True)

# randomize signal+background  events
df_total = Shuffling(df_total)
print ("Total events after shuffling: ",df_total.shape)

In [ ]:
# savePD
df_total.to_pickle('Files/MixData_PD.pkl')

<br>
<br>

## 7. Building and Training a Deep Neural Network with Keras

### Model Representation and input arrays
- __X__ : Denotes the "input" variables. Is a Numpy array of dimensions [events, variables] containing the distributions to be used as inputs to the DNN.

- **y** : Denotes the "output" or target variable that we are trying to predict. Is an array of dimension (events). Since here we want to tell  S from B (binary classification problem) y can take  only two values, 0 and 1 indicating Background or Signal respectively. (Most of what we say here will also generalize to the multiple-class case.) y is also called the label for the training example.

Our goal is, given a training set, to learn a function **h : X $\rightarrow$ Y** so that **h(x)** is a "good" predictor for the corresponding value of **y.**

<br>
Lets create the arrays described above. We are going to use only a subset of the available variables for X but you can excercise with different inputs.

In [ ]:
InputFeatures = ['lep1_pt','lep1_eta','lep1_phi','lep1_E', 'lep2_pt','lep2_eta','lep2_phi','lep2_E', 'Zll_mass','Zll_pt', 'MET']
# make the X array
X=df_total[InputFeatures].values

#make y vector
y_tmp = df_total['isSignal']

# make the event weights vector
w=df_total['FullEventWeight']

In [ ]:
# lets print the arrays before scaling
print(X)
print(y_tmp)

<br>

### Feature Scaling
We can speed up gradient descent by having each of our input values in roughly the same range. This is because minimization of the cost function with respect to the parameters will descend quickly on small ranges and slowly on large ranges, and so will oscillate inefficiently down to the optimum when the variables are very uneven.

Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual feature do not more or less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance).

The way to prevent this is to modify the ranges of our input variables so that they are all roughly the same. This can be done using the **StandardScaler** module from the **scikit-learn** (http://scikit-learn.org) library.

In [ ]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler()
X = scaler.fit_transform(X)

le = LabelEncoder()
y = le.fit_transform(y_tmp)

In [ ]:
# lets print the arrays after scaling
print(X)
print(y)

<br>

### Create train and test samples
We need to split our dataset in 3 parts in order to evaluate its performance and be able to tell how well our model generarizes to unseen data. Split dataset into multiple parts:
- **Training set**: Used to fit model parameters
- **Validation set**: Used to check performance on independent data and tune hyper parameters
- **Test set**: Final evaluation of performance after all hyper-parameters are fixed

In [ ]:
from sklearn.model_selection import train_test_split

#split into training and testing samples
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, w, train_size=0.7,random_state=123)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

<br>

### Building a DNN in Keras

Models in Keras are defined as a sequence of layers. We create a Sequential model and add 'hidden' layers.

In this example, we will use a fully-connected network structure with 3 layers.

Fully connected layers are defined using the **Dense** class. We can specify the number of neurons in the layer with  `units=width` and specify the activation function using the `Activation('relu')` argument. We need to make sure that the input layer has the right number of inputs. This can be specified when creating the first layer with the `input_dim` argument and setting it to be equal to the shape of the X_train dataset `n_dim=X_train.shape[1]`.

```python
n_dim=X_train.shape[1]
n_nodes = 32

model = Sequential()
model.add(Dense(units=n_nodes, input_dim=n_dim))
model.add(Activation('relu'))
```

Network weights are initialized from a small random number generated from a uniform distribution (‘uniform‘), in this case between 0 and 0.05 because that is the default uniform weight initialization in Keras. 

We add more layers and use the rectifier (‘relu‘) activation function for them.

```python
n_depth = 2 
for i in range(0, depth):
        model.add(Dense(width))
        model.add(Activation('relu'))
```

We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class

```python
model.add(Dense(1, activation='sigmoid'))
```

<br>
Lets put it all together:

In [ ]:
from keras.models import Model, Sequential,load_model
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.core import Dense, Activation
from keras.constraints import maxnorm

In [ ]:
def BuildDNN(N_input,width,depth):
    model = Sequential()
    model.add(Dense(units=width, input_dim=N_input) )  # First layer
    model.add(Activation('relu'))

    for i in range(0, depth):
        model.add(Dense(width))
        model.add(Activation('relu'))

    model.add(Dense(1, activation='sigmoid'))  # Output layer/node 

    return model

<br>
Now lets construct the model

In [ ]:
n_dim=X_train.shape[1]
n_nodes = 32
n_depth = 2 # number of additional hidden layers

model=BuildDNN(n_dim,n_nodes,n_depth)

Compiling the model uses the efficient numerical computation libraries (backends) such as **Theano** or **TensorFlow**.

When compiling, we must specify the loss function to use to evaluate a set of weights, the optimizer used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as `'binary_crossentropy'`. We will also use the gradient descent algorithm `rmsprop`. `adam` is another popular option.

Finally we use the classification accuracy as the metric.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

We can train or fit the model on the loaded data by calling the **fit()** function.

The training process will run for a fixed number of iterations through the dataset called **epochs**. We can also set the number of instances that are evaluated before a weight update in the network is performed, called the batch_size.

In [ ]:
# Start the training!
modelMetricsHistory = model.fit(X_train, y_train, epochs=20,batch_size=2048,validation_split=0.2, verbose=1)

In [ ]:
perf = model.evaluate(X_test, y_test, batch_size=2048)
testLoss = 'Test loss:%0.3f' % perf[0]
testAccuracy = 'Test accuracy:%0.3f' % perf[1]
print (testLoss)
print (testAccuracy)

In [ ]:
# summarize history for accuracy
plt.plot(modelMetricsHistory.history['acc'])
plt.plot(modelMetricsHistory.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.figtext(0.5, 0.5, testAccuracy, wrap=True, horizontalalignment='center', fontsize=10)
plt.savefig("Files/Accuracy.png")
plt.show()
plt.clf()
# summarize history for loss
plt.plot(modelMetricsHistory.history['loss'])
plt.plot(modelMetricsHistory.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.figtext(0.5, 0.5, testLoss, wrap=True, horizontalalignment='center', fontsize=10)
plt.savefig("Files/Loss.png")
plt.show()
plt.clf()

<br>

### Using regularization

In [ ]:
def BuildDNN_dropout(N_input,width,depth):
    model = Sequential()
    model.add(Dense(units=width, input_dim=N_input, kernel_constraint=maxnorm(3) ) )  # First layer
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    for i in range(0, depth):
        model.add(Dense(width))
        model.add(Activation('relu'))
        # Dropout randomly sets a fraction of input units to 0 at each update during training time
        # which helps prevent overfitting.
        model.add(Dropout(0.2)) 

    model.add(Dense(1, activation='sigmoid'))  # Output layer/node 

    return model

In [ ]:
model_dro=BuildDNN_dropout(n_dim,n_nodes,n_depth)
model_dro.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
# Start the training!
history_dro = model_dro.fit(X_train, y_train, epochs=20,batch_size=2048,validation_split=0.2, verbose=1)

In [ ]:
# summarize history for accuracy
plt.plot(history_dro.history['acc'])
plt.plot(history_dro.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.figtext(0.5, 0.5, testAccuracy, wrap=True, horizontalalignment='center', fontsize=10)
plt.savefig("Files/Accuracy.png")
plt.show()
plt.clf()
# summarize history for loss
plt.plot(history_dro.history['loss'])
plt.plot(history_dro.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.figtext(0.5, 0.5, testLoss, wrap=True, horizontalalignment='center', fontsize=10)
plt.savefig("Files/Loss.png")
plt.show()
plt.clf()

<br>

### Adding callbacks

In [ ]:
callbacks = [
# if we don't have an increase of the accuracy for 10 epochs, terminate training.
EarlyStopping(verbose=True, patience=10, monitor='val_loss'),
# Always make sure that we're saving the model weights with the best loss.
ModelCheckpoint('model.h5', monitor='val_loss', verbose=True, save_best_only=True)
]

model_dro =BuildDNN_dropout(n_dim,n_nodes,n_depth)
model_dro.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# Start the training!
history_dro = model_dro.fit(X_train, y_train, callbacks=callbacks, epochs=200,batch_size=2048,validation_split=0.2, verbose=1)

**Now you have the best model saved in your directory as 'model.h5'. You can then load it back and use it !**

model = load_model('model.h5')

## Metrics

In [ ]:
# list all data in history
print(history_dro.history.keys())
# summarize history for accuracy
plt.plot(history_dro.history['acc'])
plt.plot(history_dro.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()
# summarize history for loss
plt.plot(history_dro.history['loss'])
plt.plot(history_dro.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

<br>

### User-defined metrics

In [ ]:
from keras import backend

def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

model_dro =BuildDNN_dropout(n_dim,n_nodes,n_depth)

model_dro.compile(loss='binary_crossentropy', optimizer='adam', metrics=[rmse])
# Start the training!
history_dro = model_dro.fit(X_train, y_train, epochs=20,batch_size=2048,validation_split=0.2, verbose=1)

In [ ]:
# plot metrics
plt.plot(history_dro.history['rmse'])
plt.show()

<br>
<br>

## 8. DNN results
Once the training process has been completed we can ask the trained DNN to make predictions on unseen data and in this way evaluate its performance. We therefore apply the model to the **test** sample we had created before.

We use:
```python
model.predict(test_sample)
```

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report

# Generates output predictions for the input samples.
yhat_test = model_dro.predict(X_test,batch_size=2048)

print (yhat_test)

scoreBins = np.linspace(0,1.0, 30)
plt.hist(yhat_test,bins=scoreBins)

A way to say how well the model is predicting the correct result is the ROC curve. A ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. 

TPR is defined as the correctly identified signal over the sum of correctly identified signal + incorrectly rejected signal
```text
TPR = N_passing_s/(N_passing_s+ N_rejected_s)  or written as TPR = TP/(TP+FN)
```
and
```text
FPR = N_passing_b/(N_passing_b+ N_rejected_b) or FPR = FP/(FP+TN)
```

so in this case TPR is equivalent to Signal Efficiency and FPR is equivalent to Background Efficiency

The **Scikit-learn** library offers the functionality of easily getting a ROC curve. Just plug-in `y_test` and the prediction `yhat_test`.

The Area Under Curve (AUC) is another metric for the DNN performance and is actually the integral of the ROC curve.

In [ ]:
# Get 'Receiver operating characteristic' (ROC)
fpr, tpr, thresholds = roc_curve(y_test, yhat_test)

# Compute Area Under the Curve (AUC) from prediction scores
roc_auc  = auc(fpr, tpr)
print ("ROC AUC: %0.3f" % roc_auc)

plt.plot(fpr, tpr, color='darkorange',  lw=2, label='Full curve (area = %0.2f)' % roc_auc)
plt.plot([0, 0], [1, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC curves for Signal vs Background')
# plt.plot([0.038], [0.45], marker='*', color='red',markersize=5, label="Cut-based",linestyle="None")
# plt.plot([0.038, 0.038], [0,1], color='red', lw=1, linestyle='--') # same background rejection point
plt.legend(loc="lower right")
# plt.savefig("Files/ROC.pdf")
plt.show()
plt.clf()

In [ ]:
# X_train, X_test, y_train, y_test, w_train, w_test

Xtrain_signal     = X_train[y_train==1]
Xtrain_background = X_train[y_train!=1]

# Then do the same for Xtest
Xtest_signal     = X_test[y_test==1]
Xtest_background = X_test[y_test!=1]

# Get predictions of the model on these -train- samples
print ('Running model prediction on Xtrain_signal')
yhat_train_signal     = model_dro.predict(Xtrain_signal,batch_size=2048)
print ('Running model prediction on Xtrain_background')
yhat_train_background = model_dro.predict(Xtrain_background,batch_size=2048)

# Get predictions of the model on these -test- samples
print ('Running model prediction on Xtest_signal')
yhat_test_signal     = model_dro.predict(Xtest_signal,batch_size=2048)
print ('Running model prediction on Xtest_background')
yhat_test_background = model_dro.predict(Xtest_background,batch_size=2048)

In [ ]:
# Plot scores
bins=np.linspace(0.0,1.0, 40)
plt.hist(yhat_train_signal,     bins=bins, histtype='step',       lw=2, color='blue', label=[r'Signal Train'],     normed=True)
plt.hist(yhat_test_signal,      bins=bins, histtype='stepfilled', lw=2, color='cyan', alpha=0.5,  label=[r'Signal Test'],      normed=True)

plt.hist(yhat_train_background, bins=bins, histtype='step',       lw=2, color='red', label=[r'Background Train'], normed=True)
plt.hist(yhat_test_background,  bins=bins, histtype='stepfilled', lw=2, color='orange', alpha=0.5,  label=[r'Background Test'],  normed=True)

plt.ylabel('Norm. Entries')
plt.xlabel('DNN score')
# plt.yscale('log')
# plt.ylim((0,1000))
plt.legend(loc="upper center")

<br>

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
import itertools

yResult_test_cls  = np.array([ int(round(x[0])) for x in yhat_test])
theClasses = ['Background','Signal']

def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes, rotation=90)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    # plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
cnf_matrix = confusion_matrix(y_test, yResult_test_cls)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=theClasses,normalize=True, title='Normalized confusion matrix')

<br>

## K-fold cross validation

In [ ]:
InputFeatures = ['lep1_pt','lep1_eta','lep1_phi','lep1_E', 'lep2_pt','lep2_eta','lep2_phi','lep2_E', 'Zll_mass','Zll_pt', 'MET']

from sklearn.model_selection import StratifiedKFold
from collections  import Counter
from keras import backend as K


df_tot = df_total

X=df_tot[InputFeatures]
scaler = StandardScaler()
X = scaler.fit_transform(X)

Y= df_tot['isSignal']


kfold = StratifiedKFold(n_splits=4, shuffle=False, random_state=None)
cv_scores = []
ii = 1
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1.0, 100)
    
for train_index, test_index in kfold.split(X, Y):
    print('{} of KFold {}'.format(ii,kfold.n_splits))
    
    X_train_xval = np.array(X)[train_index,:]
    X_test_xval = np.array(X)[test_index,:]
    y_train_xval = np.array(Y)[train_index]
    y_test_xval = np.array(Y)[test_index]
    
    print (X_train_xval.shape[1])

    model = BuildDNN_dropout(X_train_xval.shape[1],32,3)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    # adjust the learning rate
    K.set_value(model.optimizer.lr, 0.003 )
    
    callbacks = [
            EarlyStopping(verbose=True, patience=5, monitor='val_loss'),
            ModelCheckpoint('model_kfold_'+str(ii)+'.h5', monitor='val_loss', verbose=True, save_best_only=True)
    ]

    kf_history = model.fit( X_train_xval, y_train_xval, epochs=200, batch_size=2048,
            validation_split=0.2,
            callbacks=callbacks,
            verbose=1)
    
    kf_y_prediction_test = model.predict(X_test_xval)
    score = roc_auc_score(y_test_xval,kf_y_prediction_test)
    print('ROC AUC score:',score)
    cv_scores.append(score)    
    
    # Get 'Receiver operating characteristic' (ROC)
    fpr, tpr, thresholds = roc_curve(y_test_xval, kf_y_prediction_test)
    roc_auc = auc(fpr, tpr)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
#     print(roc_auc)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.3f)' % (ii, roc_auc))

    ii += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Luck', alpha=.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',label=r'Mean ROC (AUC = %0.3f $\pm$ %0.3f)' % (mean_auc, std_auc),lw=1, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
plt.xticks(np.arange(0.0, 1.1, 0.1))
plt.yticks(np.arange(0.0, 1.1, 0.1))
plt.title('ROC curves for Signal vs Background')
plt.legend(loc="lower right",fontsize='x-small')
plt.show()
